In [13]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AmplitudeEmbedding
from sklearn.datasets import load_digits # import MNIST data set
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize ,OneHotEncoder # import a normilisation function
from sklearn.model_selection import train_test_split
import torch
from torchvision import datasets, transforms


In [14]:
np.random.seed(42) # set a seed
dev = qml.device('default.qubit', wires=13,shots=1000)
#prepare_state was useful to correct a QubitStateVector error
@qml.qnode(dev)
def prepare_state(input):
    for k in range(0,5):
        qml.QubitStateVector(input[k], wires=k)
    for l in range(6, 12):
        qml.QubitStateVector(input[l-6], wires=l)
    return qml.state()
#Making the Encoding and the dot product
def qknn(f=None, test=None, k_val=None):
    #prepare_state(X)
    for k in range(0,5):
        qml.QubitStateVector(f[k], wires=k)
    for l in range(6, 12):
        qml.QubitStateVector(test[l-6], wires=l)
    qml.templates.AmplitudeEmbedding(f, wires=range(0,6),normalize=True)
    qml.templates.AmplitudeEmbedding(test, wires=range(6,12),normalize=True)
    qml.Hadamard(wires=13) # This shall be the output
    for x in range(0,k_val):
        qml.CSWAP(wires=[13, x, x+6]) #control=13, target1, target2
    qml.Hadamard(wires=13) # This shall be the output
    out = qml.expval(qml.PauliZ(13))
    return out
    

X = load_digits(n_class=2,return_X_y=False)
#print(X)
data = torch.tensor(X.data, dtype = torch.int) 
# Convert numpy array to tensor 
outputs = torch.tensor(data).flatten(0)

total_records = 100 
test_records = int(total_records * .2) 
train_data = data[:total_records-test_records] 
test_data = data[total_records-test_records:total_records]
#print(len(train_data))
#print(len(test_data))


<ipython-input-14-4b4076c2e2a0>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = torch.tensor(data).flatten(0)


In [15]:
k_values = [1, 3, 5, 7]
prob = []

for k_val in k_values:
    out = qknn(train_data[0], test_data[0], k_val)
    prob.append(out)
    print(out)


expval(PauliZ(wires=[13]))
expval(PauliZ(wires=[13]))
expval(PauliZ(wires=[13]))
expval(PauliZ(wires=[13]))
